# Salário Mínimo Necessário - DIEESE
---
O **DIEESE** estima mensalmente o valor do salário mínimo necessário com base no **custo de vida real**, para sustentar uma família de 4 pessoas - 2 adultos e 2 crianças-, conforme a Constituição (Art. 7º, IV):

> "salário mínimo capaz de atender às suas necessidades vitais básicas e às de sua família com moradia, alimentação, educação, saúde, lazer, vestuário, higiene, transporte e previdência social."

Tendo por base esse artigo, o DIEESE parte do custo da cesta básica de alimentos mais cara entre as capitais pesquisadas (geralmente São Paulo).

O governo, no entanto, **não adota o critério do DIEESE** por questões fiscais, políticas e econômicas. O salário mínimo no Brasil é definido por lei federal e serve como piso para remunerações no país. Desde 2023, segue a fórmula:

> **Reajuste = INPC (ano anterior) + crescimento real do PIB (2 anos antes)**

Essa fórmula busca equilibrar **reposição inflacionária** e **valorização real**, respeitando limites fiscais, pois o salário mínimo impacta aposentadorias, benefícios e gastos públicos.

A Metodologia detalhada do DIEESE pode ser encontrada em: https://www.dieese.org.br/metodologia/metodologiaCestaBasica2016.pdf


In [26]:
# Bilbiotecas
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [24]:
# Visualizando html da página
url = "https://www.dieese.org.br/analisecestabasica/salarioMinimo.html"
response = requests.get(url)

pagina = BeautifulSoup(response.text, "html.parser")
print(pagina)

<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<div id="include">
<title>DIEESE - análise cesta básica - Salário mínimo nominal e necessário - junho/2025</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=8" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="Tabela do salário mínimo nominal e necessário" lang="" name="description" scheme=""/>
<meta content="DIEESE - análise cesta básica - Salário mínimo nominal e necessário - junho/2025" lang="" name="alternative" scheme=""/>
<link href="/css/geral.css" rel="stylesheet" type="text/css"/>
<link href="/favicon.ico" rel="icon" type="image/png"/>
<link href="https://www.dieese.org.br/analisecestabasica/salarioMinimo.html" rel="canonical"/>
<script src="/scripts/geral.js" type="text/javascript"></script><script src="/scripts/google.js" type="text/javascript"></script><script src="/scripts/jquery/jq

In [43]:
# Criando a tabela através dos elementos na página
linhas = pagina.find_all('tr')

dados = []
ano_atual = None

for linha in linhas:
    # Verifica se é um ano (class="subtitulo")
    if "subtitulo" in linha.get("class", []):
        ano_tag = linha.find("a")
        if ano_tag:
            ano_atual = ano_tag.get_text().strip()
    # Verifica se é uma linha de dados (exatamente 3 colunas (td))
    elif len(linha.find_all("td")) == 3 and ano_atual is not None:
        colunas = linha.find_all("td")
        mes = colunas[0].get_text(strip=True).replace("\n", "")
        salario_nominal = colunas[1].get_text(strip=True).replace("\n", "")
        salario_necessario = colunas[2].get_text(strip=True).replace("\n", "")

        dados.append({
            "Ano": ano_atual,
            "Mês": mes,
            "Salário Mínimo Nominal": salario_nominal,
            "Salário Mínimo Necessário": salario_necessario
        })

df = pd.DataFrame(dados)
df

,Ano,Mês,Salário Mínimo Nominal,Salário Mínimo Necessário
0,2025,Maio,"R$ 1.518,00","R$ 7.528,56"
1,2025,Abril,"R$ 1.518,00","R$ 7.638,62"
2,2025,Março,"R$ 1.518,00","R$ 7.398,94"
3,2025,Fevereiro,"R$ 1.518,00","R$ 7.229,32"
4,2025,Janeiro,"R$ 1.518,00","R$ 7.156,15"
...,...,...,...,...
366,1994,Novembro,"R$ 70,00","R$744,25"
367,1994,Outubro,"R$ 70,00","R$ 740,83"
368,1994,Setembro,"R$ 70,00","R$ 695,64"
369,1994,Agosto,"R$ 64,79","R$ 645,53"


In [45]:
# Salvando
df.to_csv("salario_minimo.csv", index=False)